In [2]:
#Import packages
#---------------------------------------
import sys
import os
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import matplotlib
import warnings
import random
import arviz as az
import pymc as pm
warnings.filterwarnings("ignore", category=RuntimeWarning) 

#Import your modules
#---------------------------------------
import admin_functions as adfn
import cell_decomp_func as cdfn


# Define paths
#----------------------------------------------------------------------
l_code = '/Users/dominicburrows/Dropbox/PhD/Analysis/my_scripts/GitHub/'
l_data = '/Users/dominicburrows/Dropbox/PhD/analysis/Project/'
l_fig = '/Users/dominicburrows/Dropbox/PhD/figures/'

s_code = '/cndd3/dburrows/CODE/'
s_data = '/cndd3/dburrows/DATA/'
%load_ext autoreload
sys.version

'3.11.0 | packaged by conda-forge | (main, Jan 14 2023, 12:27:40) [GCC 11.3.0]'

In [31]:
#==============================================================
def run_pyRCTD(idata, prop_vec):
#==============================================================
    mean_post = np.mean(idata.posterior['beta'][0],axis=0)
    from scipy.stats import linregress
    line_fit=linregress(np.ravel(prop_vec), np.ravel(mean_post))
    return(mean_post, line_fit.rvalue**2)

In [26]:
gamma_list = np.geomspace(5,100,20).astype(int)
gamma_list[0]=0
gamma_list[-1]=99

array([ 0,  5,  6,  8,  9, 10, 12, 15, 17, 20, 24, 28, 33, 38, 45, 53, 62,
       72, 85, 99])

In [24]:
#==============================================================
def add_noise(spots, per, a_std, g_std, e_std):
#==============================================================

    """
    This function adds different types of noise to simulated spots data. 
    
    Inputs:
        spots (np array): y x d array of simulated spot counts, where y = spots, d = genes.
        per (int): percentage of dropout genes per spot
        a_std (int): pixel shared gaussian noise - standard deviations 
        g_std (int): gene shared gaussian noise - standard deviations
        e_std (int): independent spot and gene noise - standard deviations
        
    returns:
        spots (np array): y x d array of simulated noisy spot counts, where y = spots, d = genes.
    
    """
    import numpy as np

    #Dropout a certain percentage of genes
    if per!=None:
        for i in range(spots.shape[0]):
            rand_ind = np.random.choice(np.arange(spots.shape[1]), size = int((per/100) * spots.shape[1]), replace=False) #random index for selecting
            spots[i,rand_ind]=0
    
    if e_std!=None:
        #Add random noise and make int and remove negatives
        spots = spots+np.random.exponential(e_std, (spots.shape)) ##EXPONENTIAL OR GAUSSIAN?

    if g_std!=None:
        #gamma - over each gene
        gamma = np.random.exponential(g_std, (spots.shape[1])) ##EXPONENTIAL OR GAUSSIAN?
        gamma_mat = np.asarray([gamma for i in range(spots.shape[0])]) #Repeat across columns for elementwise addition
        spots = spots+gamma_mat

    if a_std!=None:
        #alpha - over each spot
        alpha = np.random.exponential(a_std, (spots.shape[0])) ##EXPONENTIAL OR GAUSSIAN?
        alpha_mat = np.asarray([alpha for i in range(spots.shape[1])]).T #Repeat across columns for elementwise addition
        spots = spots+alpha_mat

    spots = spots.astype(int)  #REMOVE?
    spots[spots < 0] = 0
    spots +=1 #remove any zeros

    return(spots)

In [5]:
# Run alpha noise - exponential model

#Define parameters of simulated data
n_clusts = 5
n_genes = 800
n_cells = 100
rate_range = 0,40 #max and min of uniform distribution for generating rates
mode = 'gamma'
#per = 55 #percentage of dropped genes
# e_std= 0 #spot + gene noise
# g_std = 0 #gene specific noise

for g in gamma_list:
    #Simulate spot data from simulated gene expression
    spot_sim = cdfn.simulate_cell_mix(n_clusts, n_cells, n_genes).simulate_gene_exp(rate_range)
    n_spots = spot_sim.__dict__['n_spots']
    spots = spot_sim.__dict__['spots']
    ref_exp = spot_sim.__dict__['mean_exps']
    prop_vec = spot_sim.__dict__['prop_vec']
    spots = add_noise(spots, per=None, a_std=None, g_std=g, e_std=None) #add in noise

    #Simple Linear regression
    with pm.Model(coords={"celltypes": np.arange(n_clusts),
                        "spots": np.arange(n_spots),
                        "genes": np.arange(n_genes) }) as basic_model:
        #Declare data 
        mean_exp = pm.Data('mean_exp', ref_exp, mutable=False, dims=['celltypes','genes'])
        # Priors for unknown model parameters
        beta=pm.HalfNormal("beta", sigma=1, dims=['spots','celltypes'])
        lmd= pm.Deterministic('lmd', pm.math.dot(beta, mean_exp), dims=['spots','genes'])
        #Convert from proportions to counts
        N_g = pm.Data('N_g', np.sum(spots, 1).reshape(n_spots,1), mutable=False)
        #Likelihood of observed data given Poisson rates
        y=pm.Poisson("y", mu=lmd*N_g, observed=spots)

    #Run model
    with basic_model:
        basic_data=pm.sample(random_seed=1,draws=200,chains=1, discard_tuned_samples=False)


    #Poisson noise
    with pm.Model(coords={"celltypes": np.arange(n_clusts),
                        "spots": np.arange(n_spots),
                        "genes": np.arange(n_genes),
                        "1": np.arange(1) }) as Poisson_noise_model:
        #Declare data 
        mean_exp = pm.Data('mean_exp', ref_exp, mutable=False, dims=['celltypes','genes'])
        # Priors for unknown model parameters
        beta=pm.HalfNormal("beta", sigma=1, dims=['spots','celltypes']) # celltype proportions
        gamma = pm.Exponential("gamma", lam=1, dims=['1','genes']) # random noise at each spot
        lmd= pm.Deterministic('lmd', np.exp(gamma)*pm.math.dot(beta, mean_exp), dims=['spots','genes'])
        #Convert from proportions to counts
        N_g = pm.Data('N_g', np.sum(spots, 1).reshape(n_spots,1), mutable=False)

        #Likelihood of observed data given Poisson rates
        y=pm.Poisson("y", mu=lmd*N_g, observed=spots)

    with Poisson_noise_model:
        noise_data = pm.sample(random_seed=1,draws=200,chains=1, discard_tuned_samples=False)

    basic_post, basic_r2 = run_pyRCTD(basic_data, prop_vec)
    noise_post, noise_r2 = run_pyRCTD(noise_data, prop_vec)


    if len(str(g))<2:pref = '0' + str(g)
    else:pref= str(g)
    np.save(s_data + 'spatial_transcriptomics/RCTD-test-basic-exp-model_'+ mode + '-' + pref + '.npy', np.array([prop_vec, basic_post, basic_r2], dtype=object))
    np.save(s_data + 'spatial_transcriptomics/RCTD-test-noise-exp-model_' + mode + '-' + pref + '.npy', np.array([prop_vec, noise_post, noise_r2], dtype=object))
    print(g)


05 5
06 6
08 8
10 10
13 13
17 17
23 23
29 29
38 38
50 50
